In [23]:
from graphviz import Digraph
import itertools
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [24]:
Inflow = [x for x in itertools.product(('0','+'),('-','0','+'))]
Volume = [x for x in itertools.product(('0','+','max'),('-','0','+'))]
Outflow = [x for x in itertools.product(('0','+','max'),('-','0','+'))]
# Height = [x for x in itertools.product(('0','+','max'),('-','0','+'))]
# Pressure = [x for x in itertools.product(('0','+','max'),('-','0','+'))]
initial_state = (('0','0'),('0','0'),('0','0'))#,('0','0'),('0','0'))

In [25]:
def judgev(state):
    judge = True
    a = state[0]
    b = state[2]
    c = state[1]
    if a[0]=='+' and b[0]=='0' and c[1]!='+':
        judge = False
    elif a[0]=='0' and b[0]=='+' and c[1]!='-':
        judge = False
    elif a[0]=='0' and b[0]=='0' and c[1]!='0':
        judge = False
    return judge

def PP(a,b):
    judge = True
    if a[1]=='+' and b[1]!='+':
        judge = False
    elif a[1]=='-' and b[1]!='-':
        judge = False
    elif a[1]=='0' and b[1]!='0':
        judge = False
    return judge

def VCmax(a,b):
    judge = True
    if a[0]=='max' and b[0]!='max':
        judge = False
    return judge

def VC0(a,b):
    judge = True
    if a[0]=='0' and b[0]!='0':
        judge = False
    return judge

def judge_inner_state(state):
    judge = judgev(state)
    for (a,b) in model_restriction:
        for restrict in model_restriction[(a,b)]:
            judge = judge and restrict(state[a],state[b])
    return judge

In [26]:
def not_the_same(state1,state2):
    judge = True
    if state1==state2:
        judge = False
    return judge
def continuity(a,b):
    judge = True
    if a[0]=='0' and b[0]=='max':
        judge = False
    elif a[1]=='-' and b[1]=='+':
        judge = False
    return judge
def inflow_limit(a,b):
    judge = True
    if a==('0','0') and b not in [('0', '0'), ('0', '+')]:
        judge = False
    if a==('0','+') and b!=('+','+'):
        judge = False
    if a==('0','-') and b!=('0','0'):
        judge = False
    if a==('+','-') and b==('0','0'):
        judge = False
    if a==('+','0') and b not in [('+','-'),('+','+'),('+','0')]:
        judge = False
    if a==('+','+') and b not in [('+', '0'),('+', '+')]:
        judge = False
    return judge
def other_limit(a,b):
    judge = True
    if a==('0','0') and b not in [('0', '0'), ('0', '+')]:
        judge = False
    if a==('0','+') and b!=('+','+'):
        judge = False
    if a==('0','-') and b!=('0','0'):
        judge = False
    if a==('+','-') and b in [('0','0'),('max', '-'), ('max', '0'), ('max', '+')]:
        judge = False
    if a==('+','0') and b not in [('+','-'),('+','+'),('+','0')]:
        judge = False
    if a==('+','+') and b not in [('+', '0'),('+', '+'),('max', '+')]:
        judge = False
    if a==('max', '-') and b!=('+', '-'):
        judge = False
    if a==('max', '0') and b not in [('max', '0'),('max','-'),('max','+')]:
        judge = False
    if a==('max', '+') and b not in [('max', '0'),('max','+')]:
        judge = False
    return judge

def judge_inter_state(state1,state2):
    judge = not_the_same(state1,state2)
    for i in range(len(state1)):
        if i==0:
            judge = judge and inflow_limit(state1[i],state2[i])
        else:
            judge = judge and other_limit(state1[i],state2[i])
        judge = judge and continuity(state1[i],state2[i]) and continuity(state2[i],state1[i])
        if judge==False:
            break
    return judge

In [27]:
model_restriction = {(1,2):{PP,VCmax,VC0}}#,(1,3):{PP},(3,4):{PP}}
possibilities = {x for x in itertools.product(Inflow,Volume,Outflow) if judge_inner_state(x)}

In [28]:
state_list = [initial_state]
relation_set = set()

i = 0
j = 0

while j<5:
    try:
        state = state_list[i]
        for possible_state in possibilities:
            if judge_inter_state(state,possible_state):
                if possible_state not in state_list:
                    state_list.append(possible_state)
                relation_set.add((state_list.index(state),state_list.index(possible_state)))
        i += 1
    except:
        i -= 1
        j += 1

In [29]:
dot = Digraph(comment='Stategraph')
for s in state_list:
    dot.node(str(s))
    
for (x,y) in sorted(list(relation_set),key=lambda tup: tup[0]):
    dot.edge(str(state_list[x]), str(state_list[y]))

print(dot.source)

dot.render('test-output/Stategraph.gv',view=True)

// Stategraph
digraph {
	"(('0', '0'), ('0', '0'), ('0', '0'))"
	"(('0', '+'), ('0', '0'), ('0', '0'))"
	"(('+', '+'), ('0', '+'), ('0', '+'))"
	"(('+', '+'), ('+', '+'), ('+', '+'))"
	"(('+', '0'), ('+', '+'), ('+', '+'))"
	"(('+', '0'), ('+', '0'), ('+', '0'))"
	"(('+', '0'), ('+', '+'), ('max', '+'))"
	"(('+', '+'), ('+', '+'), ('max', '+'))"
	"(('+', '+'), ('max', '+'), ('max', '+'))"
	"(('+', '+'), ('+', '0'), ('+', '0'))"
	"(('+', '0'), ('max', '+'), ('max', '+'))"
	"(('+', '-'), ('+', '0'), ('+', '0'))"
	"(('+', '-'), ('+', '+'), ('max', '+'))"
	"(('+', '-'), ('+', '+'), ('+', '+'))"
	"(('+', '-'), ('max', '+'), ('max', '+'))"
	"(('+', '0'), ('+', '-'), ('+', '-'))"
	"(('+', '-'), ('+', '-'), ('+', '-'))"
	"(('+', '+'), ('+', '-'), ('+', '-'))"
	"(('+', '0'), ('+', '0'), ('max', '0'))"
	"(('+', '-'), ('+', '0'), ('max', '0'))"
	"(('+', '+'), ('+', '0'), ('max', '0'))"
	"(('+', '0'), ('max', '0'), ('max', '0'))"
	"(('+', '+'), ('max', '0'), ('max', '0'))"
	"(('+', '-'), ('max', '

'test-output/Stategraph.gv.pdf'